# This is Lesson_1 Jupyter Notebook

Below is an example of a simple Agent with some manually triggered functions in order to understand how it really acts.

The Part_1 of the Notebook is about Agent initialization and how it can act together with some manually done work we do for it.
The Part_2 of the Notebook is about how to use the Agent with the ReACT pattern.


In [5]:
# based on https://til.simonwillison.net/llms/python-react-pattern

In [6]:
#
# Part_1: Agent Initialization and Manual Function Calls
#

import openai
import re
import httpx
import os
from dotenv import load_dotenv
import os
import json


_ = load_dotenv()

# ensure the environment variable is set
print("OPENAI_API_KEY is loaded from .env file:", os.getenv("OPENAI_API_KEY") is not None)

from openai import OpenAI

OPENAI_API_KEY is loaded from .env file: True


In [3]:
client = OpenAI()

In [4]:
# needed here just to make sure the client is initialized and working
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-**************ERE>. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [32]:
# needed here just to make sure the client is initialized and working
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

In [35]:
class Agent:
    # agent can be parameterized using system message
    # so it allows user to pass a system message to the agent
    # The 'self' concept in Python is indeed similar to this in C#, but there are some key differences that make it more explicit.
    #
    # "Magic Method",
    # agent = Agent("system prompt")  --> Calls __init__
    def __init__(self, system=""):
        # we store the system message and initialize the messages list
        self.system = system # 'self' is REQUIRED
        self.messages = []   # 'self' is REQUIRED
        if self.system:
            # put the system message users provided as a system prompt piece
            self.messages.append({"role": "system", "content": system})

    # append the message to the existing message array as user's input
    #
    # "Magic Method"
    # agent = Agent("system prompt")
    # result = agent("Hello")            # Calls __call__
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})

        # Unlike C#, Python doesn't assume you're referring to instance members
        # result = execute()      # ❌ Error - function not found
        # result = self.execute() # ✅ Calls instance method
        result = self.execute()   # Instance method call
        self.messages.append({"role": "assistant", "content": result})
        return result

    # _method (single underscore) - "Internal/Protected"
    # execute is a public method to execute the agent's logic
    def execute(self):
        # Must use 'self' to access instance variables/methods
        completion = client.chat.completions.create(
                        model="gpt-4o",
                        temperature=0, # will make the output deterministic
                        messages=self.messages)
        return completion.choices[0].message.content

In [37]:
## Agentic Prompt. It is very specific to the ReACT pattern and the task.
#  I specify it in the Jupyter notebook cell.

system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [38]:
# module-level functions in the Jupyter cell, not methods of the Agent class.
# These are standalone functions (not part of any class)
def calculate(what):           # Function in Jupyter cell
    return eval(what)

# Function in Jupyter cell
def average_dog_weight(name):
    if name in "Scottish Terrier":
        return "Scottish Terriers average 20 lbs"
    elif name in "Border Collie":
        return "a Border Collies average weight is 37 lbs"
    elif name in "Toy Poodle":
        return "a toy poodles average weight is 7 lbs"
    else:
        return "An average dog weights 50 lbs"

# This is a dictionary variable in Jupyter cell
known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [39]:
# let's create an instance of the Agent class with the prompt
# and execute the agent with a question.
# system_prompt is specified above.
abot = Agent(system_prompt)

result = abot("How much does a toy poodle weigh?")

# In print here we send back what our Agent thinks.
print(result)

Thought: I should look up the average weight of a Toy Poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


In [40]:
# right now we manually call the function in order to get the dog's weight
# we use Toy Poodle because previously we asked the agent about it.
result = average_dog_weight("Toy Poodle")
print(result)

a toy poodles average weight is 7 lbs


In [42]:
# we prepare the next prompt for the agent adding result of the previous action we called manually
next_prompt = "Observation: {}".format(result)
print(next_prompt)

Observation: a toy poodles average weight is 7 lbs


In [43]:
# call the agent again with the prompt we constructed above with Toy Poodle weight
# in simple words we just merge our manual call with the agent's logic
abot(next_prompt)

'Answer: A Toy Poodle weighs an average of 7 lbs.'

In [49]:
# Pretty print with indentation
# show what messages the Agent has
print(json.dumps(abot.messages, indent=2))

[
  {
    "role": "system",
    "content": "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs"
  },
  {
    "role": "user",
    "cont

In [1]:
#
# Part_2: Using the Agent with ReACT Pattern.
#
abot = Agent(system_prompt)

# we add the new question to the agent
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

NameError: name 'Agent' is not defined